In [35]:
from google.cloud import bigquery
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

dict_uf = {"11":"Rondônia",
           "12":"Acre",
           "13":"Amazonas",
           "14":"Roraima",
           "15":"Pará",
           "16":"Amapá",
           "17":"Tocantins",
           "21":"Maranhão",
           "22":"Piauí",
           "23":"Ceará",
           "24":"Rio Grande do Norte",
           "25":"Paraíba",
           "26":"Pernambuco",
           "27":"Alagoas",
           "28":"Sergipe",
           "29":"Bahia",
           "31":"Minas Gerais",
           "32":"Espírito Santo",
           "33":"Rio de Janeiro",
           "35":"São Paulo",
           "41":"Paraná",
           "42":"Santa Catarina",
           "43":"Rio Grande do Sul",
           "50":"Mato Grosso do Sul",
           "51":"Mato Grosso",
           "52":"Goiás",
           "53":"Distrito Federal"}

dict_sexo = {'1': 'Homem', '2':'Mulher'}

dict_tipo_moradia = {'1':'Urbana','2':'Rural'}

dict_regiao_moradia = {'1':'Capital', '2':'Região Metropolitana', '4':'Outras regiões'}

dict_cor_raca = {'1':'Branca', '2':'Preta', '3':'Amarela', '4': 'Parda', '5':'Indígena', '9':'Ignorado'}

# Conexão com Big Query

In [2]:
project_id = 'brave-tea-400210'
dataset_id = 'fase_3_tech_challenge'
table_id = 'pnad-covid-19'
location = 'US'

client = bigquery.Client(project=project_id, location='US')

# Visualizações

## Distribuição da variável sexo ('A003') nas zonas rurais e urbanas ('V1022')

In [3]:
query_sql = f'''WITH cont_sexo AS (
SELECT
    uf,
    sexo,
    zona_moradia,
    COUNT(sexo) AS n_sexo,
FROM
    `{dataset_id}.{table_id}`
GROUP BY
    uf, zona_moradia, sexo
ORDER BY
    uf
)

SELECT
    uf,
    sexo,
    zona_moradia,
    n_sexo,
    ROUND(SUM(n_sexo)/SUM(n_sexo) OVER (PARTITION BY uf, zona_moradia),4) AS proportion
FROM
    cont_sexo
GROUP BY
    uf, sexo,zona_moradia, n_sexo
ORDER BY uf, sexo
'''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [4]:
df_results.dtypes

uf                Int64
sexo              Int64
zona_moradia      Int64
n_sexo            Int64
proportion      float64
dtype: object

In [5]:
df_results['uf'] = df_results['uf'].astype(str).replace(dict_uf)
df_results['sexo'] = df_results['sexo'].astype(str).replace(dict_sexo)
df_results['zona_moradia'] = df_results['zona_moradia'].astype(str).replace(dict_tipo_moradia)
df_results.head()

,uf,sexo,zona_moradia,n_sexo,proportion
0,Rondônia,Homem,Urbana,5763,0.4784
1,Rondônia,Homem,Rural,2382,0.5205
2,Rondônia,Mulher,Urbana,6283,0.5216
3,Rondônia,Mulher,Rural,2194,0.4795
4,Acre,Homem,Rural,1640,0.5273


In [6]:
fig_urbana = px.bar(df_results.loc[df_results['zona_moradia'] == 'Urbana'].sort_values(by='uf', ascending=False), x="proportion", y="uf", color="sexo", title="Ditribuição da população por sexo (Zona Urbana)",
            color_discrete_sequence=[px.colors.qualitative.G10[1], px.colors.qualitative.G10[0]],height=800, width=600, labels=dict(uf="Estado", sexo="Sexo", proportion="Proporção (%)"))

fig_urbana.update_layout(title_x=0.5, showlegend=False)
fig_urbana.add_vline(x=0.5)


fig_rural = px.bar(df_results.loc[df_results['zona_moradia'] == 'Rural'].sort_values(by='uf', ascending=False), x="proportion", y="uf", color="sexo", title="Ditribuição da população por sexo (Zona Rural)",
            color_discrete_sequence=[px.colors.qualitative.G10[1], px.colors.qualitative.G10[0]],height=800, width=600, labels=dict(uf="Estado", sexo="Sexo", proportion="Proporção (%)"))

fig_rural.update_layout(title_x=0.5, showlegend=False)
fig_rural.add_vline(x=0.5)

fig = make_subplots(rows=1, cols=2, shared_xaxes=True, subplot_titles=["Ditribuição da população de entrevistados por sexo (Zona Urbana)","Ditribuição da população de entevistados por sexo (Zona Rural)"])

# Add the first scatter plot to the first subplot
fig.add_trace(fig_urbana['data'][0], row=1, col=1)
fig.add_trace(fig_urbana['data'][1], row=1, col=1)


# Add the second scatter plot to the second subplot
fig.add_trace(fig_rural['data'][0], row=1, col=2)
fig.add_trace(fig_rural['data'][1], row=1, col=2)

fig.update_layout(barmode='stack', width=1600, height=800)
fig.add_vline(x=0.5)

plotly.offline.plot(fig, filename = './plots/br_distribuicao_populacao_sexo_zona_moradia.html', auto_play=False, auto_open=False)

fig.show()

Comparando a distribuição de homens e mulheres de acordo com a situação do domicílio (zona urbana ou rural), é perceptível uma tendência de maior presença de mulheres nas zonas urbanas e, por outro lado, uma predominância de homens nas zonas rurais de praticamente todos os estados brasileiros.

Possivelmente, isso está associado às atividades braçais presentes no campo e que são majoritariamente executadas por homens.

Do ponto de vista do COVID-19, esta análise indica que as mulheres estão mais presentes em regiões com grande concentração e circulação de pessoas e por esse motivo podem estar mais suscetíveis à infecção.

# Distribuição da população de acordo com tipo de área (Capital, RM ou Interior)

In [45]:
query_sql =f'''SELECT uf,
            (CASE
                  WHEN regiao_moradia = 1 THEN 1
                  WHEN regiao_moradia = 2 THEN 2
                  WHEN regiao_moradia = 3 THEN 2
                  WHEN regiao_moradia = 4 THEN 4
            END) as new_regiao_moradia,
            COUNT(CASE
                  WHEN regiao_moradia = 1 THEN 1
                  WHEN regiao_moradia = 2 THEN 2
                  WHEN regiao_moradia = 3 THEN 2
                  WHEN regiao_moradia = 4 THEN 4
            END) as n_regiao_moradia

            FROM `{dataset_id}.{table_id}`

            GROUP BY uf, new_regiao_moradia
            ORDER BY uf, new_regiao_moradia
            '''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [8]:
df_results['uf'] = df_results['uf'].astype(str).replace(dict_uf)
df_results['new_regiao_moradia'] = df_results['new_regiao_moradia'].astype(str).replace(dict_regiao_moradia)
df_results.head()

,uf,new_regiao_moradia,n_regiao_moradia
0,Rondônia,Capital,6062
1,Rondônia,Outras regiões,10560
2,Acre,Capital,11498
3,Acre,Outras regiões,7336
4,Amazonas,Capital,16567


In [28]:
fig_3 = px.bar(df_results.sort_values(by=['n_regiao_moradia'], ascending=False), x="uf", y="n_regiao_moradia", color="new_regiao_moradia", title="Ditribuição da população da pesquisa por região de moradia",
            labels=dict(uf="Estado", n_regiao_moradia="Nº de entrevistados", new_regiao_moradia="Região de moradia"))

fig_3.update_layout(title_x=0.5, width=1200, height=700, legend_traceorder='reversed')

title_name = fig_3['layout']['title']['text']

plotly.offline.plot(fig_3, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

fig_3.show()




# Distribuição da cor e raça

In [46]:
query_sql = f'''
with cont_raca as 
(SELECT
  uf,
  cor_raca,
  COUNT(cor_raca) as n_cor_raca
FROM `brave-tea-400210.fase_3_tech_challenge.pnad-covid-19` 
GROUP BY uf, cor_raca
ORDER BY uf, cor_raca)

SELECT
    uf,
    cor_raca,
    n_cor_raca,
    ROUND(SUM(n_cor_raca)/SUM(n_cor_raca) OVER (PARTITION BY uf),4) AS proportion_raca
FROM
    cont_raca
GROUP BY
    uf, cor_raca, n_cor_raca
ORDER BY uf, cor_raca
'''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [47]:
df_results['cor_raca'] = df_results['cor_raca'].astype(str).replace(dict_cor_raca)
df_results['uf'] = df_results['uf'].astype(str).replace(dict_uf)
df_results

,uf,cor_raca,n_cor_raca,proportion_raca
0,Rondônia,Branca,4833,0.2908
1,Rondônia,Preta,1236,0.0744
2,Rondônia,Amarela,145,0.0087
3,Rondônia,Parda,10360,0.6233
4,Rondônia,Indígena,48,0.0029
...,...,...,...,...
153,Distrito Federal,Preta,2071,0.1040
154,Distrito Federal,Amarela,195,0.0098
155,Distrito Federal,Parda,9734,0.4890
156,Distrito Federal,Indígena,51,0.0026


In [62]:
fig_cor_raca = px.bar(df_results.sort_values(by=['proportion_raca'], ascending=False), x="uf", y="proportion_raca", color="cor_raca", title="Ditribuição da população da pesquisa por cor/raça",
            labels=dict(uf="Estado", proportion_raca="Proporção da população (%)", cor_raca="Cor/Raça"))

fig_cor_raca.update_layout(title_x=0.5, width=1200, height=700)

title_name = fig_cor_raca['layout']['title']['text']
title_name = title_name.replace('/','-')

plotly.offline.plot(fig_cor_raca, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

fig_cor_raca.show()
